In [ ]:
# Task 4 – ETF Analysis (SPY, TLT, GLD)
# Simple, robust, and Colab-ready

!pip -q install yfinance --upgrade

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------
# a) Download prices (auto_adjust=True → adjusted prices live in 'Close')
# ------------------------------
tickers = ["SPY", "TLT", "GLD"]
START, END = "2022-01-01", "2023-01-01"

df = yf.download(tickers, start=START, end=END, auto_adjust=True, progress=False)

# Extract a clean price frame whether columns are MultiIndex or not
if isinstance(df.columns, pd.MultiIndex):
    prices = df['Close']              # works with auto_adjust=True
    prices = prices.dropna(how="all")
else:
    # single-index fallback (rare with multiple tickers)
    prices = df[['Close']].copy() if 'Close' in df.columns else df.copy()
    if prices.shape[1] == 1:
        prices.columns = [tickers[0]]

# ------------------------------
# b) Compute daily returns
# ------------------------------
returns = prices.pct_change().dropna()

# ------------------------------
# c) Annualized performance metrics (252 trading days)
# ------------------------------
mean_ann = returns.mean() * 252
vol_ann  = returns.std() * np.sqrt(252)
sharpe   = mean_ann / vol_ann

metrics = pd.DataFrame({
    "Mean Return (ann.)": mean_ann,
    "Volatility (ann.)": vol_ann,
    "Sharpe Ratio": sharpe
}).round(4)

print("Task 4 – Annualized Performance Metrics")
print(metrics)
print()

# ------------------------------
# d) Correlation matrix + heatmap
# ------------------------------
corr = returns.corr().round(3)
print("Task 4 – Correlation Matrix")
print(corr)

plt.figure(figsize=(6,5))
plt.imshow(corr, cmap="coolwarm", interpolation="none")
plt.xticks(range(len(corr)), corr.columns, rotation=45)
plt.yticks(range(len(corr)), corr.columns)
plt.colorbar(label="Correlation")
plt.title("Task 4: ETF Correlation Matrix (Daily Returns)")
plt.tight_layout()
plt.show()
